<a href="https://colab.research.google.com/github/vasneva/user_behavior_analysis/blob/main/user_behavior.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from matplotlib import pyplot as plt
import seaborn as sns

## Загрузка данных и их анализ

In [2]:
url = 'https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/p_UjGG_Dv8CE5Q'
full_data = pd.read_csv(url)

In [3]:
full_data.shape

(2747968, 8)

In [4]:
full_data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [5]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747968 entries, 0 to 2747967
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   date          object 
 1   event         object 
 2   purchase_sum  float64
 3   os_name       object 
 4   device_id     int64  
 5   gender        object 
 6   city          object 
 7   utm_source    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 167.7+ MB


In [6]:
# приведем дату к соответствующему формату
full_data.date = pd.to_datetime(full_data.date)

In [7]:
# проверим наличие нулевых значений
full_data.isnull().sum()

date                  0
event                 0
purchase_sum    2606585
os_name               0
device_id             0
gender                0
city                  0
utm_source            0
dtype: int64

### 1. Посчитаем число уникальных *пользователей* за февраль 2020 года


In [8]:
full_data['month'] = full_data.date.dt.strftime('%Y-%m')

In [9]:
  full_data.head(3)

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01


In [10]:
mau = full_data.query("month == '2020-02'").device_id.nunique()
print(f"MAU в феврале 2020 года составило {mau}")

MAU в феврале 2020 года составило 75032


### 2. Посчитаем количество установок в январе 2020

In [11]:
install_df = full_data \
                      .query("month =='2020-01'") \
                      .groupby('event') \
                      .agg({'device_id': 'count'}) \
                      .rename(columns={'device_id': 'actions_number'})
installation_num = install_df.loc['app_install', 'actions_number']
print(f'В январе приложение было установлено {installation_num} раз')

В январе приложение было установлено 80297 раз


### 3. Разделим пользователей на когорты по дню установки приложения и посчитаем конверсию из установки в покупку в течение 7 дней

In [12]:
install_date = full_data.query("event == 'app_install'")[['device_id', 'date']]

In [13]:
install_date.rename(columns={'date':'install_date'}, inplace=True)

In [14]:
install_date.device_id.nunique()

154597

In [15]:
full_data = full_data.merge(install_date, on='device_id', how='left')

In [16]:
full_data['days'] = (full_data.date - full_data.install_date).dt.days
full_data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,install_date,days
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01,2020-01-01,0.0
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01,2020-01-01,0.0
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01,NaT,NaN
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01,2020-01-01,0.0
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01,2020-01-01,0.0


In [17]:
cohort_volume = full_data.groupby('install_date', as_index=False).agg({'device_id':'nunique'})
cohort_volume.head()

,install_date,device_id
0,2020-01-01,3579
1,2020-01-02,3144
2,2020-01-03,2402
3,2020-01-04,1831
4,2020-01-05,1671


In [18]:
CR_df = full_data.query("event == 'purchase' & days <= 7") \
                 .groupby('install_date', as_index=False) \
                 .agg({'device_id':'nunique'}) \
                 .sort_values('device_id')

In [19]:
CR_df = CR_df.merge(cohort_volume, on='install_date', how='left')

In [20]:
CR_df.rename(columns={'device_id_x':'num_return_users', 'device_id_y':'total_users'}, inplace=True)

In [21]:
CR_df.head()

,install_date,num_return_users,total_users
0,2020-03-16,92,360
1,2020-03-17,108,431
2,2020-03-05,115,411
3,2020-03-14,120,443
4,2020-03-15,125,478


In [22]:
CR_df['retention_rate, %'] = CR_df.num_return_users / CR_df.total_users * 100

In [23]:
CR_df.sort_values('retention_rate, %')

,install_date,num_return_users,total_users,"retention_rate, %"
75,2020-03-06,807,6358,12.692671
21,2020-03-28,209,1091,19.156737
27,2020-03-22,251,1261,19.904837
22,2020-03-29,223,1117,19.964190
24,2020-03-18,235,1171,20.068318
...,...,...,...,...
83,2020-01-02,1186,3144,37.722646
89,2020-01-14,1973,5173,38.140344
87,2020-01-15,1650,4310,38.283063
67,2020-01-09,558,1424,39.185393


### 4. Выяснить с какого платного маркетингового канала пришло больше всего новых пользователей

In [24]:
full_data.groupby(['device_id', 'utm_source'], as_index=False).agg({'date':'min'}) \
         .groupby('utm_source', as_index=False).agg({'device_id':'count'}).sort_values('device_id', ascending=False)

,utm_source,device_id
0,-,52273
6,yandex-direct,40712
2,google_ads,38096
5,vk_ads,34086
3,instagram_ads,31048
1,facebook_ads,25959
4,referal,15926


Больше всего пользователей из Яндекса

### 5. Проанализировать на каком этапе воронки уходит большая часть зарегистрированных клиентов

In [25]:
full_data.groupby('event', as_index=False).agg({'device_id':'count'})

,event,device_id
0,app_install,154597
1,app_start,748705
2,choose_item,538669
3,purchase,141383
4,register,78310
5,search,708639
6,tap_basket,377665


In [26]:
registered = full_data[full_data['event'] == 'register'][['device_id', 'date']]

In [27]:
registered

,device_id,date
2669658,294193,2020-01-01
2669659,22917617,2020-01-01
2669660,15248490,2020-01-01
2669661,252062,2020-01-01
2669662,2251583,2020-01-01
...,...,...
2747963,2984778,2020-03-31
2747964,27301864,2020-03-31
2747965,1294285,2020-03-31
2747966,3010574,2020-03-31


In [28]:
registered.rename(columns={'date': 'registration_date'}, inplace=True)

In [29]:
registered.device_id.nunique()

78310

In [30]:
full_data = full_data.merge(registered, on='device_id', how='left')

In [31]:
full_data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,install_date,days,registration_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01,2020-01-01,0.0,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01,2020-01-01,0.0,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01,NaT,NaN,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01,2020-01-01,0.0,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01,2020-01-01,0.0,NaT


In [32]:
full_data = full_data.assign(registered=full_data.apply(lambda x: 'registered' if x['registration_date'] < x['date']
                                            else 'not_registered', axis=1))

In [33]:
registered_df = full_data[full_data['registered'] == 'registered']

In [34]:
registered_df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,install_date,days,registration_date,registered
15148,2020-01-02,app_start,NaN,ios,3163406,female,Moscow,instagram_ads,2020-01,2020-01-01,1.0,2020-01-01,registered
15175,2020-01-02,app_start,NaN,ios,2169108,female,Moscow,google_ads,2020-01,2020-01-01,1.0,2020-01-01,registered
15197,2020-01-02,app_start,NaN,android,4863305,male,Saint-Petersburg,vk_ads,2020-01,2020-01-01,1.0,2020-01-01,registered
15210,2020-01-02,app_start,NaN,android,15248490,male,Moscow,facebook_ads,2020-01,2020-01-01,1.0,2020-01-01,registered
15214,2020-01-02,app_start,NaN,ios,25617659,male,Moscow,vk_ads,2020-01,2020-01-01,1.0,2020-01-01,registered


In [35]:
reg_sum = registered_df.groupby('event').agg({'device_id':'nunique'})
reg_sum

,device_id
event,
app_start,40991
choose_item,37926
purchase,24880
search,40482
tap_basket,34517


In [36]:
# app_start >> search >> choose_item >> tap_basket >> purchase
step_1 = round(reg_sum.loc['search', 'device_id'] / reg_sum.loc['app_start', 'device_id'], 4) * 100
print(f'Конверсия перехода из запуска приложения в поиск {step_1}%')

Конверсия перехода из запуска приложения в поиск 98.76%


In [37]:
step_2 = round(reg_sum.loc['choose_item', 'device_id'] / reg_sum.loc['search', 'device_id'], 4) * 100
print(f'Конверсия перехода из поиска в отправку товара в корзину  {step_2}%')

Конверсия перехода из поиска в отправку товара в корзину  93.69%


In [38]:
step_3 = round(reg_sum.loc['tap_basket', 'device_id'] / reg_sum.loc['choose_item', 'device_id'], 3) * 100
print(f'Конверсия перехода из отправки товара в корзину в переход в корзину {step_3}%')

Конверсия перехода из отправки товара в корзину в переход в корзину 91.0%


In [39]:
step_4 = round(reg_sum.loc['purchase', 'device_id'] / reg_sum.loc['tap_basket', 'device_id'], 4) * 100
print(f'Конверсия перехода из перехода в корзину в подтверждение покупки {step_4}%')

Конверсия перехода из перехода в корзину в подтверждение покупки 72.08%


Конверсия в покупку из перехода в корзину самая низкая и составляет 72,08%

### 6. Выяснить, пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку

In [40]:
first_start = full_data[full_data['event'] == 'app_start'] \
      .groupby(['device_id', 'utm_source'], as_index=False).agg({'date':'min'}) \
      .groupby('utm_source').agg({'device_id':'nunique'})
first_start

,device_id
utm_source,
-,52268
facebook_ads,25957
google_ads,38092
instagram_ads,31045
referal,15924
vk_ads,34079
yandex-direct,40707


In [41]:
purchsed_users = full_data[full_data['event'] == 'purchase'] \
        .groupby(['device_id','utm_source'], as_index=False).agg({'date':'min'}) \
        .groupby('utm_source').agg({'device_id':'nunique'})
purchsed_users

,device_id
utm_source,
-,16598
facebook_ads,9017
google_ads,11339
instagram_ads,10762
referal,6362
vk_ads,12364
yandex-direct,12028


In [42]:
facebook_cr = round(purchsed_users.loc['facebook_ads', 'device_id'] / first_start.loc['facebook_ads', 'device_id'] * 100, 2)
print(f'Конверсия в первую покупку для пользователей пришедших из Facebook {facebook_cr }')

Конверсия в первую покупку для пользователей пришедших из Facebook 34.74


In [43]:
google_cr = round(purchsed_users.loc['google_ads', 'device_id'] / first_start.loc['google_ads', 'device_id'] * 100, 2)
print(f'Конверсия в первую покупку для пользователей пришедших из Google {google_cr }')

Конверсия в первую покупку для пользователей пришедших из Google 29.77


In [44]:
instagram_cr = round(purchsed_users.loc['instagram_ads', 'device_id'] / first_start.loc['instagram_ads', 'device_id'] * 100, 2)
print(f'Конверсия в первую покупку для пользователей пришедших из Instagram {instagram_cr}')

Конверсия в первую покупку для пользователей пришедших из Instagram 34.67


In [45]:
vk_cr = round(purchsed_users.loc['vk_ads', 'device_id'] / first_start.loc['vk_ads', 'device_id'] * 100, 2)
print(f'Конверсия в первую покупку для пользователей пришедших из ВКонтакте {vk_cr}')

Конверсия в первую покупку для пользователей пришедших из ВКонтакте 36.28


In [46]:
yandex_cr = round(purchsed_users.loc['yandex-direct', 'device_id'] / first_start.loc['yandex-direct', 'device_id'] * 100, 2)
print(f'Конверсия в первую покупку для пользователей пришедших из Яндекс {yandex_cr}')

Конверсия в первую покупку для пользователей пришедших из Яндекс 29.55


In [47]:
referal_cr = round(purchsed_users.loc['referal', 'device_id'] / first_start.loc['referal', 'device_id'] * 100, 2)
print(f'Конверсия в первую покупку для пользователей пришедших по реферальной программе {referal_cr}')

Конверсия в первую покупку для пользователей пришедших по реферальной программе 39.95


Самую низкую конверсию в первую покупку имеют пользователи пришедшие из Яндекс

### 7. Пользователи, пришедшие с какого калана, имеют наибольший медианный первый чек?

In [48]:
purchases = full_data[full_data['event'] == 'purchase'][['date', 'device_id', 'purchase_sum', 'utm_source']]
purchases = purchases.sort_values('date').drop_duplicates('device_id')

In [49]:
purchases.drop(columns='device_id').groupby('utm_source').median(numeric_only=True).sort_values(by='purchase_sum', ascending=False)

,purchase_sum
utm_source,
-,398.5
referal,395.5
instagram_ads,393.5
vk_ads,393.0
yandex-direct,392.5
google_ads,390.5
facebook_ads,389.0


Наибольший медианный первый чек имеют пользователи из реферальной программы

### 8. Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI?

ROMI = 100 * (Доходы - Расходы на маркетинг) / Расходы на маркетинг

**Данные по затратам на рекламу:**

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

In [50]:
yandex_direct = 10_491_707
google_ads = 10_534_878
facebook_ads = 8_590_498
instagram_ads = 8_561_626
vk_ads = 9_553_531

In [51]:
revenue = full_data.groupby('utm_source')['purchase_sum'].sum().to_frame()
revenue

,purchase_sum
utm_source,
-,21449749.5
facebook_ads,12249901.0
google_ads,12868276.0
instagram_ads,14546969.0
referal,8837044.5
vk_ads,16389652.5
yandex-direct,13915368.0


In [52]:
facebook_romi = round((100 * (revenue.loc['facebook_ads', 'purchase_sum'] - facebook_ads) / facebook_ads), 2)
print(f'ROMI для Facebook: {facebook_romi}')

ROMI для Facebook: 42.6


In [53]:
google_romi = round((100 * (revenue.loc['google_ads', 'purchase_sum'] - google_ads) / google_ads), 2)
print(f'ROMI для Google: {google_romi }')

ROMI для Google: 22.15


In [54]:
instagram_romi = round((100 * (revenue.loc['instagram_ads', 'purchase_sum'] - instagram_ads) / instagram_ads), 2)
print(f'ROMI для Instagram: {instagram_romi}')

ROMI для Instagram: 69.91


In [55]:
vk_romi = round((100 * (revenue.loc['vk_ads', 'purchase_sum'] - vk_ads) / vk_ads), 2)
print(f'ROMI для ВКонтакте: {vk_romi}')

ROMI для ВКонтакте: 71.56


In [56]:
yandex_romi = round((100 * (revenue.loc['yandex-direct', 'purchase_sum'] - yandex_direct) / yandex_direct), 2)
print(f'ROMI для Яндекс: {yandex_romi}')

ROMI для Яндекс: 32.63


Самый высокий ROMI имеет ВКонтакте